TO DO:
8SDMA tests not matched (118 total)
159 asap results not matched (~1419 total)

In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import timedelta
pd.options.display.max_colwidth = 100
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)

In [2]:
data_dir = '../../Data/noah/'

patient_file  = os.path.join(data_dir, "hashed_patient_data.csv")
history_file = os.path.join(data_dir, "hashed_history_data.csv") 
sdma_file      = os.path.join(data_dir, "hashed_SDMA_data.csv")
asap_file  = os.path.join(data_dir, "hashed_asap_data.csv")

patient = pd.read_csv(patient_file, encoding="latin-1")
history = pd.read_csv(history_file, encoding="latin-1")
sdma = pd.read_csv(sdma_file, encoding="latin-1")
asap = pd.read_csv(asap_file, encoding="latin-1", skipinitialspace=True)

patient:  patient code 
history: ID (clinic_ref) 
sdma: patient code
asap: clinic_ref

Plan:
- join patient to each history record via patient code (output: same size as history)
- scheme to match dates for history, sdma and asap??

#### join patient and history

In [ ]:
print(patient.shape)
print(history.shape)

In [ ]:
patient_history = pd.merge(patient, history, how='outer', left_on="Patient Code", right_on="Patient code")
print(patient_history.shape)
# remove records where there is no history ID (patient had no history record)
patient_history = patient_history[patient_history['ID'].notnull()]
print(patient_history.shape)
# remove records where date is null
patient_history = patient_history[patient_history['Date'].notnull()]
print(patient_history.shape)
# drop some columns we don't need
patient_history.drop(inplace=True, columns = ['Species','Unnamed: 0_x', 'Last vaccination', 'Next vaccination','Label 3 data', 'No. found', 'vacc reminder', 'time', 'created', 'VaccIntMonths', 'Label 1 data', 'Label 4 data', 'Vaccine Name', 'LatestWeight', 'modified', 'VaccIntDays', 'Label 2 data', 'Contact last', 'Practice name', 'OverDue', 'NextVaccine Name', 'Unnamed: 0_y', 'found_count', 'HL7code'])
print(patient_history.shape)

#### remove multiple history records by merging their data into one record

In [ ]:
for index, row in patient_history.iterrows(): #_copy:
    date = row['Date']
    cat = row['Patient code']
    # for this cat on this date are there multiple history entries?
    # search in history_patient records
    thisDate = patient_history[patient_history['Date'] == date]
    thisDateAndCat = thisDate[thisDate['Patient code'] == cat]
    if len(thisDateAndCat) > 1:
        # yes there are multiple history entries (merge then remove by index)
        indices = list(thisDateAndCat.index)
        # combine repeated histories onto first one
        for index in indices[-1:]:
            patient_history.loc[indices[0]] = patient_history.loc[indices[0]].fillna(patient_history.loc[index])
        # drop repeated records by index
        patient_history.drop(indices[-1:], inplace = True)
# see how many records there are now
print(patient_history.shape)

#### join sdma by patient code and date

In [ ]:
# make new df and add SDMA column and fill with nans to start
patient_history_sdma = pd.DataFrame(patient_history)
patient_history_sdma["sdma"] = np.nan
# remove column
sdma.drop(inplace=True, columns = ['Unnamed: 0'])
# make Dates the right format
sdma['Date'] = pd.to_datetime(sdma['Date'], infer_datetime_format=True)
# drop duplicate sdma records
sdma.drop_duplicates(subset=['Patient Code', 'Date'], keep='first', inplace=True)
# reset the index
sdma.reset_index(inplace=True)
print(patient_history.shape)
print(sdma.shape)

In [ ]:
sdma.duplicated(subset=['Patient Code', 'Date']).value_counts() #.unique()

In [ ]:
# make a copy of sdma
sdma_copy = pd.DataFrame(sdma)
# make sure date format is correct still
sdma_copy['Date'] = pd.to_datetime(sdma_copy['Date'], format='%Y-%m-%d') 
# make Dates the right format
patient_history_sdma['Date'] = pd.to_datetime(patient_history_sdma['Date'], format='%Y-%m-%d') 

In [ ]:
# try scheme

dayOffset = [0,-1, 1, -2, 2, -3, 3, -4, 4, -5, 5, -6, 6, -7, 7, -8, 8, -9, 9, -10, 10]
for offset in dayOffset:
    print("Day offset: " + str(offset))

    matched = []
    for index, row in sdma_copy.iterrows():
        date = row['Date'] + pd.DateOffset(offset)
        cat = row['Patient Code']
        # search in history_patient records
        thisDate = patient_history_sdma[patient_history_sdma['Date'] == date]
        thisDateAndCat = thisDate[thisDate['Patient code'] == cat]
        if len(thisDateAndCat) == 0:
            print('index '+str(index)+' has no matching record')
            print(date)
            print(cat)
        elif len(thisDateAndCat) > 1:
            print('index '+str(index)+' more than 1!') # don't expect this cos we merged same data history records
            print(thisDateAndCat)
        elif len(thisDateAndCat) == 1:
            # add SDMA to main df 
            ind = patient_history_sdma[(patient_history_sdma['Patient code'] == cat) & (patient_history_sdma['Date'] == date)].index
            patient_history_sdma.loc[ind, 'sdma'] = row['SDMA']
            matched.append(index)
            print('index '+str(index)+' matched')
        else:
            print('index '+str(index)+' WEIRD')

    # drop all records that were matched this time
    sdma_copy.drop(sdma_copy.index[matched], inplace=True)
    # reset the index
    sdma_copy.reset_index(drop=True, inplace=True)

In [ ]:
# match by hand 
#thisCat = 'edc131d1c97255c2fea0e77564b28b67a24ca02b910f98e9fb9c91783212bedd'
#patient_history_sdma[patient_history_sdma['Patient Code'] == thisCat][['Date', 'BCS', 'Weight', 'Pathology', 'Sex','sdma']]
#sdma_copy[sdma_copy['Patient Code'] == 'thisCat']

In [ ]:
# MANUALLY MATCH THE REST (MUST DO THIS AFTER SCHEME DUE TO INDEX BEING RESET EACH TIME!)

# switch day and month
patient_history_sdma.loc[13605, 'sdma'] = sdma_copy.loc[2, 'SDMA']
sdma_copy.drop(2, inplace=True)

# switch day and month
patient_history_sdma.loc[10666, 'sdma'] = sdma_copy.loc[4, 'SDMA']
sdma_copy.drop(4, inplace=True)

# switch day and month
patient_history_sdma.loc[17631, 'sdma'] = sdma_copy.loc[5, 'SDMA']
sdma_copy.drop(5, inplace=True)

# switch day and month, 1 day off
patient_history_sdma.loc[5561, 'sdma'] = sdma_copy.loc[6, 'SDMA']
sdma_copy.drop(6, inplace=True)

# switch day and month
patient_history_sdma.loc[18458, 'sdma'] = sdma_copy.loc[8, 'SDMA']
sdma_copy.drop(8, inplace=True)

# switch day and month
patient_history_sdma.loc[3551, 'sdma'] = sdma_copy.loc[11, 'SDMA']
sdma_copy.drop(11, inplace=True)

# switch day and month
patient_history_sdma.loc[10469, 'sdma'] = sdma_copy.loc[12, 'SDMA']
sdma_copy.drop(12, inplace=True)

# switch day and month
patient_history_sdma.loc[5314, 'sdma'] = sdma_copy.loc[13, 'SDMA']
sdma_copy.drop(13, inplace=True)

# switch day and month
patient_history_sdma.loc[17600, 'sdma'] = sdma_copy.loc[1, 'SDMA']
sdma_copy.drop(1, inplace=True)

# switch day and month
patient_history_sdma.loc[17616, 'sdma'] = sdma_copy.loc[14, 'SDMA']
sdma_copy.drop(14, inplace=True)

# switch day and month, 1 day off
patient_history_sdma.loc[18439, 'sdma'] = sdma_copy.loc[15, 'SDMA']
sdma_copy.drop(15, inplace=True)

# switch day and month
patient_history_sdma.loc[10232, 'sdma'] = sdma_copy.loc[16, 'SDMA']
sdma_copy.drop(16, inplace=True)

# switch day and month
patient_history_sdma.loc[17125, 'sdma'] = sdma_copy.loc[17, 'SDMA']
sdma_copy.drop(17, inplace=True)

# switch day and month
patient_history_sdma.loc[18844, 'sdma'] = sdma_copy.loc[18, 'SDMA']
sdma_copy.drop(18, inplace=True)

# switch day and month
patient_history_sdma.loc[12612, 'sdma'] = sdma_copy.loc[19, 'SDMA']
sdma_copy.drop(19, inplace=True)

# switch day and month
patient_history_sdma.loc[18913, 'sdma'] = sdma_copy.loc[20, 'SDMA']
sdma_copy.drop(20, inplace=True)

# switch day and month
patient_history_sdma.loc[18699, 'sdma'] = sdma_copy.loc[21, 'SDMA']
sdma_copy.drop(21, inplace=True)

# switch day and month
patient_history_sdma.loc[11036, 'sdma'] = sdma_copy.loc[23, 'SDMA']
sdma_copy.drop(23, inplace=True)

# switch day and month
patient_history_sdma.loc[11034, 'sdma'] = sdma_copy.loc[24, 'SDMA']
sdma_copy.drop(24, inplace=True)

# switch day and month
patient_history_sdma.loc[14272, 'sdma'] = sdma_copy.loc[25, 'SDMA']
sdma_copy.drop(25, inplace=True)

# switch day and month
patient_history_sdma.loc[19372, 'sdma'] = sdma_copy.loc[26, 'SDMA']
sdma_copy.drop(26, inplace=True)

# switch day and month
patient_history_sdma.loc[15026, 'sdma'] = sdma_copy.loc[28, 'SDMA']
sdma_copy.drop(28, inplace=True)

#
patient_history_sdma.loc[13212, 'sdma'] = sdma_copy.loc[29, 'SDMA']
sdma_copy.drop(29, inplace=True)

# switch day and month
patient_history_sdma.loc[19477, 'sdma'] = sdma_copy.loc[30, 'SDMA']
sdma_copy.drop(30, inplace=True)

# 2nd June 2018. Joined to 30th June which had ASAP.There was 2nd July w/ no path.
patient_history_sdma.loc[14292, 'sdma'] = sdma_copy.loc[3, 'SDMA']
sdma_copy.drop(3, inplace=True)

#day/month opposite
patient_history_sdma.loc[17500, 'sdma'] = sdma_copy.loc[33, 'SDMA']
sdma_copy.drop(33, inplace=True)


In [ ]:
print( str(len(sdma_copy)) + ' sdma tests unmatched')

In [ ]:
#patient_history_sdma[patient_history_sdma['Date of Birth'].isnull()] #263 patients with nan dob
#patient_history_sdma[patient_history_sdma['Sex'].isnull()] # 208 nans
#patient_history_sdma[patient_history_sdma['Desexed?'].isnull()] # 338 nans

#### join asap by clinic_ref and date

In [ ]:
# DO SOME CLEANING FIRST

# check for duplicate entries
asap.duplicated(subset=['clinic_ref', 'date']).value_counts()

# remove white space from column names
asap.rename(columns=lambda x: x.strip(), inplace=True)

# remove unwanted characters
asap = asap.replace('Ã¥Ã', '', regex=True)
asap = asap.replace('\*','',regex=True) 
asap = asap.replace('\<', '', regex=True)
asap = asap.replace('\>', '', regex=True)

# remove column
asap.drop(inplace=True, columns = ['Unnamed: 0', 'Unnamed: 1', 'owner', 'animal_id']) #'filename', 

# change species_breed column to breed column
asap.rename(columns={'species_breed':'breed'}, inplace=True)
asap = asap.replace('Feline', '', regex=True)
asap = asap.replace(',', '', regex=True)
asap = asap.replace('feline', '', regex=True)
asap['breed'] = asap['breed'].str.strip()
# drop some sneaky snouts
asap.drop(asap[asap.breed == "Canine                         Labrador Cross"].index, inplace=True)
asap.drop(asap[asap.breed == "Canine                        Staffy cross"].index, inplace=True)

In [ ]:
# make new df and add new columns
patient_history_sdma_asap = pd.DataFrame(patient_history_sdma)
extraCols = list(asap.columns)
extraCols.remove('clinic_ref')
extraCols.remove('date')
patient_history_sdma_asap = pd.concat([patient_history_sdma_asap,pd.DataFrame(columns=extraCols)], sort=True)

In [ ]:
patient_history_sdma_asap.head()

In [ ]:
asap.head(3)['date']

In [ ]:
# make Dates the right format
asap['date'] = pd.to_datetime(asap['date'], format='%d/%m/%y') 
# reset the index
asap.reset_index(inplace=True)
# do this one again just in case
patient_history_sdma_asap['Date'] = pd.to_datetime(patient_history_sdma_asap['Date'], format='%Y-%m-%d')
print(patient_history_sdma.shape)
print(asap.shape)

In [ ]:
def matching_scheme(dayOffsets, testResults_df, dateStr_res, catStr_res, original_df, dateStr_orig, catStr_orig, combined_df, testList):

    for offset in dayOffset:
        print("Day offset: " + str(offset))

        matched = []
        for index, row in testResults_df.iterrows():
            date = row[dateStr_res] + pd.DateOffset(offset)
            cat = row[catStr_res]
            # search in original records
            thisDate = original_df[original_df[dateStr_orig] == date]
            thisDateAndCat = thisDate[thisDate[catStr_orig] == cat]
            if len(thisDateAndCat) == 0:
                print('index '+str(index)+' has no matching record')
                print(date)
                print(cat)
            elif len(thisDateAndCat) > 1:
                print('index '+str(index)+' more than 1!') # don't expect this cos we merged same data history records
                print(thisDateAndCat)
            elif len(thisDateAndCat) == 1:
                
                # add results to main df 
                ind = combined_df[(combined_df[catStr_orig] == cat) & (combined_df[dateStr_orig] == date)].index
                for test in testList:
                       combined_df.loc[ind, test] = row[test]
                
                matched.append(index)
                print('index '+str(index)+' matched')
          
        else:
                print(str(len(thisDateAndCat)))
                print('index '+str(index)+' WEIRD')

        # drop all records that were matched this time
        testResults_df.drop(testResults_df.index[matched], inplace=True)
        # reset the index
        testResults_df.reset_index(drop=True, inplace=True)

In [ ]:
# use scheme
dayOffsets = [0,-1, 1, -2, 2, -3, 3, -4, 4, -5, 5, -6, 6, -7, 7, -8, 8, -9, 9, -10, 10]
asap_copy = pd.DataFrame(asap)
matching_scheme(dayOffsets, asap_copy, 'date', 'clinic_ref', patient_history_sdma, 'Date', 'ID', patient_history_sdma_asap, extraCols)

In [ ]:
# see how many cats with asap are found in the history by clinic_ref
# ( tried using filename instead of clinic ref but it didn't match any extra)

cats = 0
nocat = 0
for index, row in asap_copy.iterrows():
        cat = row['clinic_ref']
        # search cat in history
        thisCat = patient_history_sdma_asap[patient_history_sdma_asap['ID'] == cat]
        if len(thisCat) == 0:
            nocat += 1
        elif len(thisCat) == 1:
            cats += 1
            print(thisCat['ID'])
        else:
            print('something weird:' +str(thiCat))

In [ ]:
print(cats)

In [ ]:
# tried to match the 35 cats with history records by hand
# many of them only had 1 record and dates didn't match so not useful.  Did about 15 then stopped.
#thisCat = 'd1c949da20d5919de92892814310098c24b5a971d5e7823368d2351f20a32c0c'
#asap_copy[asap_copy['clinic_ref'] == thisCat][['age_sex', 'date', 'breed']]
#patient_history_sdma_asap[patient_history_sdma_asap['ID'] == thisCat][['Date', 'Sex', 'breed']]

def copy_data(from_here, to_here, these_tests):
    for test in these_tests:
        patient_history_sdma_asap.loc[to_here, test] = asap_copy.loc[from_here, test]
    asap_copy.drop(from_here, inplace=True)
    
# MATCH BY HAND

# ~ 1 mnth difference
#copy_data(154, 17679, extraCols)

#### save merged data to csv

In [ ]:
# write to csv
patient_history_sdma_asap.to_csv(data_dir + 'merged_data_31Dec18.csv')